In [1]:
import sys
sys.path.append('../')

from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv(), override=True)

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores.chroma import Chroma

from load_document import load_document
from document_db import DocumentDB

In [2]:
embedding = OpenAIEmbeddings()

vectorstore = Chroma(
                persist_directory="../data/document_db/state_of_the_union",
                embedding_function=embedding,
            )

db = DocumentDB("../data/document_db", vectorstore=vectorstore)
retriever = db.as_retriever()

In [3]:
docs = load_document("./files/state_of_the_union.txt", chunk_it=True, chunk_size=1000, chunk_overlap=100)

In [4]:
db.clean()

{'num_added': 0, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [5]:
db.upsert_documents(docs)

{'num_added': 43, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 0}

In [6]:
results = retriever.invoke("Judge Ketanji Brown Jackson")
results[0].page_content

'We cannot let this happen. \n\nTonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'

In [7]:
docs[10].page_content="modified content"

In [8]:
db.upsert_documents(docs)

{'num_added': 1, 'num_updated': 0, 'num_skipped': 42, 'num_deleted': 1}

In [9]:
db.delete_documents(['./files/state_of_the_union.txt'])

{'num_added': 1, 'num_updated': 0, 'num_skipped': 0, 'num_deleted': 43}